In [1]:
%pip install git+https://github.com/openai/CLIP.git
%pip install ultralytics
%pip install deep_sort_realtime
%pip install opencv-python-headless

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0eb7htll
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-0eb7htll
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=ee696b07d68c11b4cc566c8b6a9fadfccf791ba134494196236ddf9a1ca237d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqr9chim/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 144.9 MB/s eta 0:00:00


In [24]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow
model  = YOLO("/content/drive/MyDrive/Colab Notebooks/yolo11m-pose.pt") #put the yolo model directly or if you downloaded one put it or you put it in googledrive

cap = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/cam7.avi")#put the directory of the video here
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Use 'avc1' codec for mp4 (works better in Colab)
out = cv2.VideoWriter("/content/output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)) # the video will be created in google colab files automatic

tracker = DeepSort(
    max_age=10,
    nn_budget=100,
    embedder='clip_RN50',
    embedder_gpu=True,
    max_iou_distance=0.7,
    n_init=3
)

SKELETON = [
    (5, 7), (7, 9),     # left arm
    (6, 8), (8, 10),    # right arm
    (11, 13), (13, 15), # left leg
    (12, 14), (14, 16), # right leg
    (5, 6), (11, 12),   # shoulders, hips
    (5, 11), (6, 12),   # torso
    (0, 5), (0, 6),     # nose to shoulders
    (1, 0), (2, 0),     # eyes to nose
    (3, 1), (4, 2)      # ears to eyes
]


while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.5)
    detections = []

    for result in results:
      for box in result.boxes:
          cls = int(box.cls[0].item())
          if cls == 0:  # person class
              bx1, by1, bx2, by2 = box.xyxy[0].tolist()  # bounding box
              conf = box.conf[0].item()
              w, h = bx2 - bx1, by2 - by1
              detections.append(((bx1, by1, w, h), conf, cls))

      keypoints = result.keypoints.xy
      confidences = result.keypoints.conf

      for person_idx, person in enumerate(keypoints):
          person_conf = confidences[person_idx]

          # draw keypoints
          for i, (x, y) in enumerate(person):
              if person_conf[i] > 0.7:
                  cv2.circle(frame, (int(x), int(y)), 3, (0,255,255), -1)

          # draw skeleton lines
          for (p1, p2) in SKELETON:
              if person_conf[p1] > 0.7 and person_conf[p2] > 0.7:
                  x3, y3 = person[p1].tolist()
                  x4, y4 = person[p2].tolist()
                  cv2.line(frame, (int(x3), int(y3)), (int(x4), int(y4)), (0,255,0), 2)

      '''   this to  determin whether someone fell or not using the pose
      # fall detection check
      ls_y = person[5][1].item()
      rs_y = person[6][1].item()
      lh_y = person[11][1].item()
      rh_y = person[12][1].item()
      lk_y = person[13][1].item()
      rk_y = person[14][1].item()
      la_y = person[15][1].item()
      ra_y = person[16][1].item()

      if (rh_y >= la_y or lh_y >= ra_y or
          lk_y >= la_y  or rk_y  >= ra_y  or
          ls_y  >= la_y  or rs_y  >= ra_y ):
          cv2.rectangle(frame, (int(bx1), int(by1)), (int(bx2), int(by2)), (255,0,0), 2)
          cv2.putText(frame, "SOMEONE FELL", (int(bx1), int(by1)-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
      else:
          cv2.rectangle(frame, (int(bx1), int(by1)), (int(bx2), int(by2)), (0,255,0), 2)
          cv2.putText(frame, str(person_idx), (int(bx1), int(by1)-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2) '''


    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1,y1,x2,y2 = map(int, ltrb)

        height = y2-y1
        width = x2-x1

        # this determine whether someone fell or standing using the boundring box

        if height-width<0:
          cv2.rectangle(frame, (x1,y1), (x2,y2), (255,0,0), 2)
          cv2.putText(frame, "SOMEONE FELL", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

        else:
          cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
          cv2.putText(frame, str(track_id), (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)


    out.write(frame)

out.release()
cap.release()
cv2.destroyAllWindows()





IndentationError: unindent does not match any outer indentation level (<tokenize>, line 74)